In [6]:
import pandas as pd
import networkx as nx
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder # pip install scikit-multilearn
from skmultilearn.ensemble import LabelSpacePartitioningClassifier
from skmultilearn.problem_transform import LabelPowerset
from sklearn.svm import SVC
from community import community_louvain  # pip install python-louvain

In [4]:
def build_cooccurrence_graph(y):
    # Создание матрицы ко-встречаемости
    graph_builder = LabelCooccurrenceGraphBuilder(
        weighted=True, 
        include_self_edges=False
    )
    
    # Построение взвешенного графа
    edge_map = graph_builder.transform(y)
    
    # Конвертация в NetworkX граф
    graph = nx.Graph()
    for (u, v), weight in edge_map.items():
        graph.add_edge(u, v, weight=weight)
    
    return graph


def cluster_labels(graph):
    # Применение алгоритма Louvain для модулярности
    partition = community_louvain.best_partition(graph)
    
    # Группировка лейблов по кластерам
    clusters = {}
    for label, cluster_id in partition.items():
        clusters.setdefault(cluster_id, []).append(label)
    
    return list(clusters.values())


def train_model(X, y, clusters):
    # Инициализация базового классификатора
    base_classifier = SVC(
        kernel='rbf', 
        probability=True, 
        class_weight='balanced'
    )
    
    # Создание мульти-лабельного классификатора
    model = LabelSpacePartitioningClassifier(
        classifier=LabelPowerset(classifier=base_classifier),
        clusterer=clusters  # Результаты кластеризации
    )
    
    # Обучение модели
    model.fit(X, y)
    return model

In [23]:
data = pd.read_feather("../0. Data/wide_data.feather")

X = data.loc[:, ~data.columns.isin(["HL_1", "HL_2", "HL_3", "HL_4", "HL_5", "HL_6", "id"])]
# y = MultiLabelBinarizer().fit_transform(data.loc[:, "HL_1":"HL_6"])
y = data.loc[:, "HL_1":"HL_6"]

# Построение графа
label_graph = build_cooccurrence_graph(y)

# Кластеризация
clusters = cluster_labels(label_graph)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Обучение модели
model = train_model(X_train, y_train, clusters)

AttributeError: 'DataFrame' object has no attribute 'getformat'

In [22]:
y

array([[1, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1]])

In [ ]:


# Предсказание топ-3 кодов
probabilities = model.predict_proba(X_test)
top3_codes = get_top3_predictions(probabilities)  # Кастомная функция ранжирования

Новое

In [99]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder
from skmultilearn.ensemble import LabelSpacePartitioningClassifier
from skmultilearn.problem_transform import LabelPowerset
from community import community_louvain
import networkx as nx

# 1. Кастомный кластеризатор для фиксированных кластеров
class FixedClusterer:
    def __init__(self, clusters):
        self.clusters = clusters
        
    def fit_predict(self, X, y):
        return self.clusters


# 1. Загрузка и подготовка данных
def load_data(filename):
    data = pd.read_feather(filename)
    
    # Преобразование в бинарные метки (топ-3 кода)
    y = pd.DataFrame(np.zeros((len(data), 6), dtype=bool), columns=[f'HL_{i+1}' for i in range(6)])
    for i, row in data.iterrows():
        top3 = row[['HL_1', 'HL_2', 'HL_3', 'HL_4', 'HL_5', 'HL_6']].nlargest(3).index
        y.loc[i, top3] = True
    
    X = data.drop(columns=['HL_1', 'HL_2', 'HL_3', 'HL_4', 'HL_5', 'HL_6'])
    return X, y

# 2. Построение графа совместной встречаемости
def build_cooccurrence_graph(y):
    y_np = y.values
    graph_builder = LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False)
    edge_map = graph_builder.transform(y_np)
    n_labels = y_np.shape[1]
    adj_matrix = np.zeros((n_labels, n_labels), dtype=np.float64)
    for (i, j), weight in edge_map.items():
        adj_matrix[i, j] = weight
    return nx.from_numpy_array(adj_matrix)

# 3. Кластеризация меток
def get_label_clusters(graph):
    partition = community_louvain.best_partition(graph, resolution=1.35)
    clusters = {}
    for node, cluster_id in partition.items():
        clusters.setdefault(cluster_id, []).append(node)
    return list(clusters.values())

# 4. Обучение модели
def train_model(X_train, y_train, clusters):
    base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    clusterer = FixedClusterer(clusters)  # Используем кастомный кластеризатор
    return LabelSpacePartitioningClassifier(
        classifier=LabelPowerset(classifier=base_classifier),
        clusterer=clusterer
    ).fit(X_train, y_train)

In [100]:
X, y = load_data("../0. Data/wide_data.feather")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
graph = build_cooccurrence_graph(y_train)
clusters = get_label_clusters(graph)
model = train_model(X_train, y_train.values, clusters)  # Преобразуем y_train в numpy array
accuracy = model.score(X_test, y_test)
print(f'Model Accuracy: {accuracy:.4f}')

Model Accuracy: 0.0882


In [103]:
answ = model.predict(X_test)

answ

<List of Lists sparse matrix of dtype 'int32'
	with 204 stored elements and shape (68, 6)>

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import community as community_louvain  # Импорт модуля для Louvain-кластеризации
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# === 1. Загрузка данных ===
# Заменяем путь на указанный вами:
df = pd.read_feather("../0. Data/wide_data.feather")

# === 2. Подготовка данных: выделение признаков и меток ===
# Предполагается, что столбцы HL_1 ... HL_6 содержат бинарные значения для меток
X = df.drop(columns=["HL_1", "HL_2", "HL_3", "HL_4", "HL_5", "HL_6"])
Y = df[["HL_1", "HL_2", "HL_3", "HL_4", "HL_5", "HL_6"]].values

# === 3. Построение графа совместного появления меток ===
# Вычисляем матрицу совместного появления
cooccurrence_matrix = np.dot(Y.T, Y)

# Создаем пустой граф
G = nx.Graph()
labels = ["HL_1", "HL_2", "HL_3", "HL_4", "HL_5", "HL_6"]

# Добавляем ребра между метками, если они встречаются вместе хотя бы в одном экземпляре
for i in range(len(labels)):
    for j in range(i + 1, len(labels)):
        if cooccurrence_matrix[i, j] > 0:
            G.add_edge(labels[i], labels[j], weight=cooccurrence_matrix[i, j])

# === 4. Кластеризация меток методом Louvain ===
# Метод best_partition возвращает словарь, где ключ — имя узла (метки), а значение — номер кластера
partition = community_louvain.best_partition(G, resolution=1.35)
print("Кластеризация меток (Louvain):")
for label, cluster in partition.items():
    print(f"  {label}: кластер {cluster}")

# === 5. Разбиение данных на обучающую и тестовую выборки ===
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# === 6. Обучение модели multi-label классификации ===
# Используем метод Label Powerset, который объединяет набор меток в одну комбинированную метку
classifier = LabelPowerset(RandomForestClassifier(n_estimators=100, random_state=42))
classifier.fit(X_train, Y_train)

# Предсказание на тестовой выборке
Y_pred = classifier.predict(X_test).toarray()

# === 7. Оценка модели ===
accuracy = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average="micro")

print(f"\nAccuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")


Кластеризация меток (Louvain):
  HL_1: кластер 0
  HL_2: кластер 0
  HL_3: кластер 0
  HL_4: кластер 0
  HL_5: кластер 0
  HL_6: кластер 0


d:\Programs\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multilabel-indicator targets

In [115]:
Y_pred

array([[1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
 